In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import Input, LocallyConnected1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [4]:
# The number of steps within one time segment
TIME_PERIODS = 400
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 400
N_FEATURES = 12

In [5]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'Class_label']


In [6]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [7]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [8]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [9]:
 pd.options.display.float_format = "{:,.5f}".format

In [10]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [11]:
verbose, epochs, batch_size = 0, 120, 32

In [12]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 40.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [13]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [14]:
# import os
path='/media/naveen/nav/mat_codes/nina_DB4_codes/naveen_prep_nlw/sub_wise_process_TT/'

In [15]:
def conv_block(input_data,n_channel):
    inputs=input_data
    print(inputs.shape)
    l_y=[]
    for i in range(0,n_channel):
        ip=inputs[:,:,i]
        print('ip_shape',ip.shape)
#         ip1=tf.convert_to_tensor(ip, dtype=tf.float32)
        ip=tf.reshape(ip,(tf.shape(ip)[0],400,1))
        print('ip_shape',ip.shape)
        x = Conv1D(64, kernel_size=3,kernel_initializer="he_normal",kernel_regularizer=l1(1e-04), \
               padding='same',input_shape=ip.shape)(ip)
        print('conv1_o',x.shape)
        x = BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x = Activation('relu')(x)
        x= Conv1D(filters=64, kernel_size=3,padding="same", kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1)(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        y= Dropout(0.5)(x)
        l_y.append(y)
    y_stack=tf.stack(l_y, axis=2)   
    return y_stack

In [16]:
n_length =  400
n_depth=12
n_channel=12
n_outputs=52

In [17]:
def generate_model():
    inputs = Input(shape=(n_length,n_depth),name="main")
    p=conv_block(inputs,12)
    print(p.shape)
    print(type(p))
#     print(y.shape)
    y=Flatten()(p)
    print(y.shape)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(128, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    outputs=Dense(n_outputs, activation='softmax')(y)
    print(outputs.shape)
#     wei_mscnn_model = Model(inputs,sub_inputs], outputs)
    print('inputs_shape',inputs.shape)
    final_model = Model(inputs,outputs,name="wei_mscnn_model")
    return final_model   

In [18]:
if __name__ == "__main__":
    model = generate_model()

(None, 400, 12)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
(None, 400, 12, 64)
<class 'tensorflow.python.framework.ops.Tensor'>
(None, 307200)
(None, 52)
inputs_shape (None, 400, 12)


In [19]:
for i in range(1,11):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test)
    df_Test.head(5)
#     feature standardization
    scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])
    df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])
    df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])
    
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    # print(y_train)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
    print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_length =  400
    n_depth=12
    n_channel=12
    x_train = x_train.reshape(x_train.shape[0], n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0],  n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
    adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
#     sgd=optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=False, name='SGD')
    checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB4_20X10/checkpoint.hdf5'
#     model.load_weights(checkpoint_filepath) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB4_20X10/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)
    test_mean=statistics.mean(test_acc)
    print('test_mean for %d subjects:',i)
    print(test_mean)

/media/naveen/nav/mat_codes/nina_DB4_codes/naveen_prep_nlw/sub_wise_process_TT/S1_tr.csv
Number of columns in the dataframe: 13
Number of rows in the dataframe: 2080000

/media/naveen/nav/mat_codes/nina_DB4_codes/naveen_prep_nlw/sub_wise_process_TT/S1_tt.csv
Number of columns in the dataframe: 13
Number of rows in the dataframe: 1040000

x_train shape:  (5199, 400, 12)
5199 training samples
y_train shape:  (5199,)
num_time_periods 400
num_sensors 12
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0]
x_train shape: (400, 12)
input_shape: (400, 12)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
New y_train shape:  (5199, 

Epoch 1/120
163/163 [==============================] - ETA: 0s - loss: 6.2324 - accuracy: 0.0616
Epoch 00001: val_accuracy improved from -inf to 0.19931, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB4_20X10/checkpoint.hdf5
163/163 [==============================] - 148s 908ms/step - loss: 6.2324 - accuracy: 0.0616 - val_loss: 5.0986 - val_accuracy: 0.1993 - lr: 0.0010
Epoch 2/120
163/163 [==============================] - ETA: 0s - loss: 4.6579 - accuracy: 0.1789
Epoch 00002: val_accuracy improved from 0.19931 to 0.24971, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB4_20X10/checkpoint.hdf5
163/163 [==============================] - 63s 388ms/step - loss: 4.6579 - accuracy: 0.1789 - val_loss: 4.0282 - val_accuracy: 0.2497 - lr: 0.0010
Epoch 3/120
163/163 [==============================] - ETA: 0s - loss: 3.7857 - accuracy: 0.2468
Epoch 00003: val_accuracy improved from 0.24971 to 0.28742, saving model to /media/na

163/163 [==============================] - ETA: 0s - loss: 0.7991 - accuracy: 0.9465
Epoch 00053: val_accuracy did not improve from 0.34706
163/163 [==============================] - 55s 336ms/step - loss: 0.7991 - accuracy: 0.9465 - val_loss: 4.1538 - val_accuracy: 0.2855 - lr: 1.0000e-04
Epoch 54/120
163/163 [==============================] - ETA: 0s - loss: 0.7764 - accuracy: 0.9454
Epoch 00054: val_accuracy did not improve from 0.34706
163/163 [==============================] - 55s 335ms/step - loss: 0.7764 - accuracy: 0.9454 - val_loss: 4.1886 - val_accuracy: 0.2820 - lr: 1.0000e-04
Epoch 55/120
163/163 [==============================] - ETA: 0s - loss: 0.7506 - accuracy: 0.9508
Epoch 00055: val_accuracy did not improve from 0.34706
163/163 [==============================] - 54s 334ms/step - loss: 0.7506 - accuracy: 0.9508 - val_loss: 4.1247 - val_accuracy: 0.2828 - lr: 1.0000e-04
Epoch 56/120
163/163 [==============================] - ETA: 0s - loss: 0.7238 - accuracy: 0.9552
Epo

Epoch 80/120
163/163 [==============================] - ETA: 0s - loss: 0.4403 - accuracy: 0.9733
Epoch 00080: val_accuracy did not improve from 0.34706
163/163 [==============================] - 54s 333ms/step - loss: 0.4403 - accuracy: 0.9733 - val_loss: 4.2108 - val_accuracy: 0.2740 - lr: 1.0000e-05
Epoch 81/120
163/163 [==============================] - ETA: 0s - loss: 0.4384 - accuracy: 0.9736
Epoch 00081: val_accuracy did not improve from 0.34706
163/163 [==============================] - 54s 333ms/step - loss: 0.4384 - accuracy: 0.9736 - val_loss: 4.2564 - val_accuracy: 0.2759 - lr: 1.0000e-05
Epoch 82/120
163/163 [==============================] - ETA: 0s - loss: 0.4372 - accuracy: 0.9710
Epoch 00082: val_accuracy did not improve from 0.34706
163/163 [==============================] - 54s 333ms/step - loss: 0.4372 - accuracy: 0.9710 - val_loss: 4.2489 - val_accuracy: 0.2774 - lr: 1.0000e-05
Epoch 83/120
163/163 [==============================] - ETA: 0s - loss: 0.4347 - accurac

Epoch 107/120
163/163 [==============================] - ETA: 0s - loss: 0.3974 - accuracy: 0.9781
Epoch 00107: val_accuracy did not improve from 0.34706
163/163 [==============================] - 54s 334ms/step - loss: 0.3974 - accuracy: 0.9781 - val_loss: 4.2049 - val_accuracy: 0.2786 - lr: 1.0000e-05
Epoch 108/120
163/163 [==============================] - ETA: 0s - loss: 0.4002 - accuracy: 0.9781
Epoch 00108: val_accuracy did not improve from 0.34706
163/163 [==============================] - 55s 338ms/step - loss: 0.4002 - accuracy: 0.9781 - val_loss: 4.3143 - val_accuracy: 0.2824 - lr: 1.0000e-05
epoch_number 10
train accuracy and validation accuracy 0.4316214621067047 0.3470565676689148
82/82 [==============================] - 6s 72ms/step - loss: 3.0374 - accuracy: 0.3471
test_accuracy 0.3470565676689148
[0.3470565676689148]
test_mean for %d subjects: 1
0.3470565676689148
/media/naveen/nav/mat_codes/nina_DB4_codes/naveen_prep_nlw/sub_wise_process_TT/S2_tr.csv
Number of columns 

...

x_test shape:  (2599, 400, 12)
2599 testing samples
y_test shape:  (2599,)
x_train shape:  (5199, 400, 12)
x_test shape:  (2599, 400, 12)
n_outputs 52
Model: "wei_mscnn_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main (InputLayer)               [(None, 400, 12)]    0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 400)]        0           main[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_strided_slice_2 (Te [(None, 400)]        0           main[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_strided_slice_4 (Te 

Epoch 1/120
163/163 [==============================] - ETA: 0s - loss: 4.7360 - accuracy: 0.1118
Epoch 00001: val_accuracy improved from -inf to 0.21047, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB4_20X10/checkpoint.hdf5
163/163 [==============================] - 123s 755ms/step - loss: 4.7360 - accuracy: 0.1118 - val_loss: 3.7439 - val_accuracy: 0.2105 - lr: 0.0010
Epoch 2/120
163/163 [==============================] - ETA: 0s - loss: 3.6712 - accuracy: 0.2114
Epoch 00002: val_accuracy improved from 0.21047 to 0.24933, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB4_20X10/checkpoint.hdf5
163/163 [==============================] - 60s 367ms/step - loss: 3.6712 - accuracy: 0.2114 - val_loss: 3.4454 - val_accuracy: 0.2493 - lr: 0.0010
Epoch 3/120
163/163 [==============================] - ETA: 0s - loss: 3.3733 - accuracy: 0.2751
Epoch 00003: val_accuracy did not improve from 0.24933
163/163 [=====================

Epoch 27/120
163/163 [==============================] - ETA: 0s - loss: 1.7414 - accuracy: 0.8113
Epoch 00027: val_accuracy did not improve from 0.27241
163/163 [==============================] - 61s 372ms/step - loss: 1.7414 - accuracy: 0.8113 - val_loss: 5.0619 - val_accuracy: 0.2378 - lr: 0.0010
Epoch 28/120
163/163 [==============================] - ETA: 0s - loss: 1.6601 - accuracy: 0.8334
Epoch 00028: val_accuracy did not improve from 0.27241
163/163 [==============================] - 61s 377ms/step - loss: 1.6601 - accuracy: 0.8334 - val_loss: 5.1141 - val_accuracy: 0.2297 - lr: 0.0010
Epoch 29/120
163/163 [==============================] - ETA: 0s - loss: 1.6257 - accuracy: 0.8402
Epoch 00029: val_accuracy did not improve from 0.27241
163/163 [==============================] - 58s 353ms/step - loss: 1.6257 - accuracy: 0.8402 - val_loss: 5.0519 - val_accuracy: 0.2301 - lr: 0.0010
Epoch 30/120
163/163 [==============================] - ETA: 0s - loss: 1.5974 - accuracy: 0.8430
Ep

Epoch 81/120
163/163 [==============================] - ETA: 0s - loss: 0.5273 - accuracy: 0.9796
Epoch 00081: val_accuracy did not improve from 0.27241
163/163 [==============================] - 57s 353ms/step - loss: 0.5273 - accuracy: 0.9796 - val_loss: 5.1162 - val_accuracy: 0.2305 - lr: 1.0000e-05
Epoch 82/120
163/163 [==============================] - ETA: 0s - loss: 0.5331 - accuracy: 0.9752
Epoch 00082: val_accuracy did not improve from 0.27241
163/163 [==============================] - 56s 345ms/step - loss: 0.5331 - accuracy: 0.9752 - val_loss: 5.1146 - val_accuracy: 0.2316 - lr: 1.0000e-05
Epoch 83/120
163/163 [==============================] - ETA: 0s - loss: 0.5306 - accuracy: 0.9765
Epoch 00083: val_accuracy did not improve from 0.27241
163/163 [==============================] - 57s 347ms/step - loss: 0.5306 - accuracy: 0.9765 - val_loss: 5.0830 - val_accuracy: 0.2312 - lr: 1.0000e-05
Epoch 84/120
163/163 [==============================] - ETA: 0s - loss: 0.5317 - accurac

KeyboardInterrupt: 

In [ ]:
statistics.mean(test_acc)